In [1]:
import gpflow
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_style("white")
sns.set_context("paper", font_scale=1.4)
import sys
sys.path.append('../')
from GPCollabPrefAct import real_occ_collab_gpmc

RuntimeError: module compiled against API version 0xb but this version of numpy is 0xa

RuntimeError: module compiled against API version 0xb but this version of numpy is 0xa

In [2]:
num_latent_gps = 3

In [3]:
action_data_file = '../data/raw_data/main_actions_data.xlsx'

In [4]:
main_data = pd.read_excel(open(action_data_file, 'rb'), sheet_name = 0)

In [5]:
pref_rel_actions = main_data[main_data['Want more view'] == 0]
pref_rel_actions = pref_rel_actions[pref_rel_actions['Want more privacy'] == 0]

In [6]:
all_occ_wi_prev = pref_rel_actions['work plane illuminance before action']
all_occ_wi_cur = pref_rel_actions['work plane illuminance after action']

In [7]:
concat_all_occ_wi = np.hstack([all_occ_wi_cur, all_occ_wi_prev])
wi_mean = np.mean(concat_all_occ_wi) 
wi_std = np.std(concat_all_occ_wi)

In [23]:
wi_min = np.min(concat_all_occ_wi)
wi_max = np.max(concat_all_occ_wi)

In [24]:
print 'max WI:'
print wi_max
print 'min WI:'
print wi_min
print 'mean WI:'
print wi_mean
print 'std mean:'
print wi_std

max WI:
2409.6
min WI:
160.9
mean WI:
924.908411215
std mean:
570.281070006


In [9]:
norm1 = (concat_all_occ_wi - 1200)/1200

In [10]:
np.sort(norm1)

array([-0.86591667, -0.85816667, -0.83333333, -0.803     , -0.80291667,
       -0.80116667, -0.79375   , -0.79291667, -0.78916667, -0.7775    ,
       -0.7745    , -0.77      , -0.76866667, -0.76175   , -0.76166667,
       -0.75833333, -0.74433333, -0.74066667, -0.7205    , -0.7185    ,
       -0.71466667, -0.70833333, -0.70733333, -0.7045    , -0.70433333,
       -0.70416667, -0.70275   , -0.70225   , -0.7       , -0.69233333,
       -0.69166667, -0.68433333, -0.68133333, -0.68116667, -0.67991667,
       -0.67833333, -0.677     , -0.67666667, -0.67533333, -0.66825   ,
       -0.66183333, -0.6595    , -0.65875   , -0.65683333, -0.65533333,
       -0.63641667, -0.63333333, -0.625     , -0.62475   , -0.6205    ,
       -0.61991667, -0.60416667, -0.58816667, -0.58533333, -0.58333333,
       -0.58333333, -0.58116667, -0.58016667, -0.57991667, -0.56725   ,
       -0.56416667, -0.55483333, -0.55441667, -0.55083333, -0.5505    ,
       -0.54758333, -0.545     , -0.545     , -0.54425   , -0.54

In [11]:
occ_ids = np.unique(np.array(pref_rel_actions[['Occupant ID']]))

In [33]:
occ_ids

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18])

In [12]:
pref_rel_actions_data_list = []
pref_rel_actions_data_norm_list = []
for occ in occ_ids:
    wi_prev = np.array(pref_rel_actions[pref_rel_actions.iloc[:,0] == occ][['work plane illuminance before action']])
    wi_after = np.array(pref_rel_actions[pref_rel_actions.iloc[:,0] == occ][['work plane illuminance after action']])
    pref_rel_actions_mat = np.hstack([wi_prev, wi_after])
    pref_rel_actions_norm_mat = (pref_rel_actions_mat - wi_mean)/wi_std
    
    pref_rel_actions_data_list.append(pref_rel_actions_mat)
    pref_rel_actions_data_norm_list.append(pref_rel_actions_norm_mat)

In [13]:
#all_x_grid = np.linspace(100, 2400, 50)
#all_x_grid_norm = (all_x_grid - wi_mean)/wi_std

In [14]:
X_unique = np.unique(concat_all_occ_wi)
X_grid = np.sort(X_unique)
X_grid_norm = (X_grid - wi_mean)/wi_std

# Collaborative Preference Learning Model Setup

In [15]:
latent_gps_kerns_list = [gpflow.kernels.RBF(1)]*num_latent_gps

In [16]:
#def __init__(self, actions_list, kerns_list, X_grid)
m = real_occ_collab_gpmc.GPCollabPrefLearnGPMC(pref_rel_actions_data_norm_list, latent_gps_kerns_list, X_grid_norm)

/Users/nimishawalgaonkar/anaconda2/lib/python2.7/site-packages/numpy/core/numeric.py:190: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  a = empty(shape, dtype, order)


In [18]:
for i in xrange(len(latent_gps_kerns_list)):
    m.kerns_list[i].lengthscales.prior = gpflow.priors.Gamma(1., 1.)
    m.kerns_list[i].variance.prior = gpflow.priors.Gamma(1., 1.)

In [19]:
m.W.prior = gpflow.priors.Gaussian(0,1)

In [20]:
m.optimize(maxiter=5000) # start near MAP

      fun: array([ 641.96527781])
 hess_inv: <669x669 LbfgsInvHessProduct with dtype=float64>
      jac: array([ -6.34629530e-04,  -6.23013795e-04,  -4.52083968e-04,
        -7.16530688e-04,  -1.13631915e-04,  -1.17188031e-04,
        -2.92258336e-04,  -1.88688255e-04,  -2.19990053e-04,
        -3.66183409e-04,  -2.04281580e-04,  -1.87185690e-04,
        -1.29295947e-04,  -1.88222522e-04,  -1.08812055e-04,
        -1.19699845e-04,  -2.32559559e-04,  -1.36726598e-04,
        -2.79803792e-04,  -1.35146812e-04,  -1.19694538e-04,
        -1.53767011e-04,  -1.06602973e-04,  -1.07833105e-04,
        -7.54777865e-05,  -6.92830875e-05,  -6.71686546e-05,
        -5.91071594e-05,  -7.16116301e-05,  -1.25530898e-04,
        -8.14513792e-05,  -8.68056608e-05,  -4.40062267e-05,
        -2.17015605e-05,  -1.09471662e-05,  -7.52090287e-06,
        -2.17996164e-06,   3.77887197e-06,   4.80774676e-06,
         1.46375764e-05,   4.76615820e-05,   5.21449371e-05,
         4.27041795e-05,   4.26173500e-05

In [21]:
m

In [22]:
samples = m.sample(1000, verbose=True, epsilon=0.02, thin = 2, burn = 500, Lmax = 40)

burn-in sampling started
Iteration:  100 	 Acc Rate:  97.0 %
Iteration:  200 	 Acc Rate:  100.0 %
Iteration:  300 	 Acc Rate:  99.0 %
Iteration:  400 	 Acc Rate:  100.0 %
Iteration:  500 	 Acc Rate:  100.0 %
burn-in sampling ended
Iteration:  100 	 Acc Rate:  98.0 %
Iteration:  200 	 Acc Rate:  99.0 %
Iteration:  300 	 Acc Rate:  97.0 %
Iteration:  400 	 Acc Rate:  100.0 %
Iteration:  500 	 Acc Rate:  99.0 %
Iteration:  600 	 Acc Rate:  100.0 %
Iteration:  700 	 Acc Rate:  100.0 %
Iteration:  800 	 Acc Rate:  100.0 %
Iteration:  900 	 Acc Rate:  100.0 %
Iteration:  1000 	 Acc Rate:  100.0 %
Iteration:  1100 	 Acc Rate:  98.0 %
Iteration:  1200 	 Acc Rate:  100.0 %
Iteration:  1300 	 Acc Rate:  98.0 %
Iteration:  1400 	 Acc Rate:  100.0 %
Iteration:  1500 	 Acc Rate:  100.0 %
Iteration:  1600 	 Acc Rate:  100.0 %
Iteration:  1700 	 Acc Rate:  99.0 %
Iteration:  1800 	 Acc Rate:  100.0 %
Iteration:  1900 	 Acc Rate:  100.0 %
Iteration:  2000 	 Acc Rate:  100.0 %


# Predict latent GPs Hs and Ws - Fully Bayesian

In [25]:
min_grid = 100.
max_grid = 2500
num_grid = 20

Xt = np.linspace(min_grid, max_grid, 2*num_grid)
Xt_norm = (Xt - 1200)/1200
Xt1 = np.linspace(min_grid, max_grid, num_grid)
Xt2 = np.linspace(min_grid, max_grid, num_grid)

Xtt1, Xtt2 = np.meshgrid(Xt1, Xt2)
X_grid = np.zeros(shape =(np.ravel(Xtt1).shape[0],2))
X_grid[:,0] = np.ravel(Xtt1) 
X_grid[:,1] = np.ravel(Xtt2)

X_grid_norm = (X_grid - 1200)/1200
num_feat = X_grid_norm.shape[1]/2

In [27]:
# Finding the posterior preference probabilities at the grid points
X_grid_norm_concat = np.vstack([X_grid_norm[:,:num_feat], X_grid_norm[:,num_feat:]])

In [28]:
def multi_normal_samples(mu, std, num_samples):
    """
    Sampling from a normal distribution with multiple mean and std
    """
    samples = np.array([])
    for i in xrange(mu.shape[0]):
        sample = np.random.normal(mu[i], std[i], num_samples)
        samples = np.append(samples, sample)
    return samples

def h_all_samples(mu_list, var_list):
    """
    Utility samples for all concerned occupants
    Inputs:
    mu_list : list of mean utility values for all occupants len(mu_list) = num of occupants
    var_list : list of variance associated with utility values for all occupants
    """
    h_samples_mat = np.zeros((len(mu_list), mu_list[0].shape[0]))
    for i in xrange(len(mu_list)):
        mu = mu_list[i]
        #print mu.shape
        var = var_list[i]
        #print var.shape
        std = np.sqrt(var)
        h_sample = multi_normal_samples(mu, std, 1)
        #print u_sample
        h_samples_mat[i] = h_sample
    return h_samples_mat

In [29]:
def utility_samples_all_occupants(mu_list, var_list):
    """
    Utility samples for all concerned occupants
    Inputs:
    mu_list : list of mean utility values for all occupants len(mu_list) = num of occupants
    var_list : list of variance associated with utility values for all occupants
    """
    u_samples_mat = np.zeros((len(mu_list), mu_list[0].shape[0]))
    for i in xrange(len(mu_list)):
        mu = mu_list[i]
        #print mu.shape
        var = var_list[i]
        #print var.shape
        std = np.sqrt(var)
        u_sample = multi_normal_samples(mu, std, 1)
        #print u_sample
        u_samples_mat[i] = u_sample
    return u_samples_mat

In [30]:
from scipy.stats import norm

u_list = []
h_list = []
w_list = []
p_list = []
for s in samples:
    m.set_state(s)
    mu_list, var_list = m.predict_h(X_grid_norm_concat)
    h_all_occ = utility_samples_all_occupants(mu_list, var_list)
    W = m.W.value 
    u = np.dot(W, h_all_occ)
    num_datapoints = u.shape[1]
    u1 = u[:,:num_datapoints/2]
    u2 = u[:,num_datapoints/2:]
    u_diff = u2 - u1
    p = norm.cdf(u_diff)
    
    h_list.append(h_all_occ)
    w_list.append(W)
    u_list.append(u)
    p_list.append(p)

In [31]:
h_all_mat = np.asarray(h_list)
w_all_mat = np.asarray(w_list)
u_all_mat = np.asarray(u_list)
p_all_mat = np.asarray(p_list)

In [32]:
p_all_mean_mat = np.mean(p_all_mat, axis = 0)
p_all_std_mat = np.std(p_all_mat, axis = 0)

# Plot and save pref probabilities for all occupants

In [44]:
mean_levels = np.linspace(0,1.)
std_levels = np.linspace(0,.4)

for occ in occ_ids:
    i = occ - 1
    
    X_previous = np.array(pref_rel_actions[pref_rel_actions.iloc[:,0] == occ][['work plane illuminance before action']])
    X_current = np.array(pref_rel_actions[pref_rel_actions.iloc[:,0] == occ][['work plane illuminance after action']])
  
    #X_current = actions_data[:,1]
    #X_previous = actions_data[:,0]
    
    plt.figure(figsize=(12,10))
    mean_reshape = p_all_mean_mat[i,:].reshape((num_grid, num_grid), order='F')
    c = plt.contourf(Xt1,Xt1,mean_reshape.T,
                     mean_levels, cmap=plt.cm.Blues)
    plt.scatter(X_previous,X_current, marker = '.', c = 'r', s = 200, alpha = 1)
    plt.colorbar(c)
    plt.xlabel('Before Action')
    plt.ylabel('After Action')
    plt.title('Mean Predicted Preference Probability ; Occupant:' + str(i))
   
    plt.savefig(('../data/real_data/results/collab/CM'
                 + 'O' + str(occ) + '_h_' + str(num_latent_gps) + '_.png'), dpi = 300)
    plt.close()
    plt.figure(figsize=(12,10))
    std_reshape = p_all_std_mat[i,:].reshape((num_grid, num_grid), order='F')
    
    c_v = plt.contourf(Xt1,Xt1,std_reshape.T,
                 std_levels, cmap=plt.cm.Blues)
    
    plt.scatter(X_previous,X_current, marker = '.', c = 'r', s = 200, alpha = 1)
    plt.colorbar(c_v)
    plt.xlabel('Before Action')
    plt.ylabel('After Action')
    plt.title('Std Predicted Preference Probability ; Occupant:' + str(i))
    plt.savefig(('../data/real_data/results/collab/CV'
                 + 'O' + str(occ) + '_h_' + str(num_latent_gps) + '_.png'), dpi = 300)
    plt.close()
    